# ***Install***

In [ ]:
!pip install pytorch-lightning pytorch-forecasting
!pip install shap
!pip show pytorch-forecasting
!pip show pytorch-lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install torchmetrics


# ***Imports***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import shap

In [ ]:
import torch
from pytorch_lightning import Trainer
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_lightning import LightningModule
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.metrics import CrossEntropy

# ***Load Data***

In [ ]:
data = pd.read_csv("/content/bank-full.csv", sep=';')
data['date'] = pd.date_range(start='1/1/2010', periods=len(data), freq='D')

# ***EDA***

In [ ]:
print(data.head())
print(data.info())
print(data['y'].value_counts())
sns.countplot(x='y', data=data)
plt.title("Target Class Distribution")
plt.show()

# ***Preprocessing for TFT***

**Encode categorical features**

In [ ]:
cat_vars = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

for var in cat_vars:
    data[var] = data[var].astype("category")

data["campaign"] = data["campaign"].astype("category")


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data["y"] = label_encoder.fit_transform(data["y"])


**train-valadit approach**

In [ ]:
data = data.sort_values("date")
data["time_idx"] = (data["date"] - data["date"].min()).dt.days

training_cutoff = data["time_idx"].max() - 1000
training = TimeSeriesDataSet(
    data[data.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="y",
    group_ids=["campaign"],
    max_encoder_length=30,
    max_prediction_length=1,
    static_categoricals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=cat_vars,
    time_varying_unknown_reals=["age", "balance", "duration", "pdays", "previous"],
    target_normalizer=NaNLabelEncoder(),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)


validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=64, num_workers=0)

# ***Model Training***

In [ ]:
import torch
from pytorch_lightning import Trainer
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.data import TimeSeriesDataSet
import torch.nn as nn

# Assuming 'train_dataloader' and 'val_dataloader' are properly set up

# Create the TemporalFusionTransformer model with the correct configuration
tft = TemporalFusionTransformer.from_dataset(
    training,  # 'training' should be a TimeSeriesDataSet object that you've already defined
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Check if GPU is available and initialize the Trainer
trainer = Trainer(
    max_epochs=10,
    gradient_clip_val=0.1,
    devices=1 if torch.cuda.is_available() else None,  # Use GPU if available, otherwise CPU
    accelerator="gpu" if torch.cuda.is_available() else "cpu",  # Specify accelerator
)

# Fit the model with the training and validation dataloaders
trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


# ***Interpretation***

**Variable importance**

In [ ]:
interpretation = tft.interpret_output(tft.predict(val_dataloader))
tft.plot_interpretation(interpretation)

In [ ]:
tft.save_model("tft_bank_model.pt")